In [16]:
import py2neo

graph = py2neo.Graph(host='localhost', user='', password='')

In [29]:
import pandas as pd

#query ="MATCH (t:Type:File) RETURN count(t)"
query ="MATCH (c1:Class)-[:EXTENDS]->(c2:Type) RETURN c1.fqn, c2.fqn LIMIT 20"
#query = "MATCH (t:Type)-[:DECLARES]->(m:Method) WHERE t.fqn STARTS WITH 'kieker' RETURN t.name as Type RETURN t.name as Type, m.name AS Method, m.incomingCalls AS Calls, m.duration AS Duration"
df = pd.DataFrame(graph.run(query).data())

In [30]:
df.head()

,c1.fqn,c2.fqn
0,org.argouml.application.ArgoVersion,java.lang.Object
1,org.argouml.application.helpers.ApplicationVer...,java.lang.Object
2,org.argouml.application.LoadModules,java.lang.Object
3,org.argouml.moduleloader.ModuleLoader2,java.lang.Object
4,org.argouml.application.Main$1,org.argouml.cognitive.AbstractCognitiveTranslator
